# 모델 저장 및 배포
### 검증을 마친 모델을 Object Storage에 저장하고 KServe를 이용하여 모델을 배포하는 실습입니다.

## 1) 패키지 설치
- 모델 저장 및 배포에 필요한 패키지를 설치합니다.

In [ ]:
%pip -q install boto3 botocore

## 2) Object Storage에 모델 저장
- Kakaocloud Object Storage에 모델을 저장합니다.
- `S3_ACCESS_KEY`, `S3_SECRET_KEY` 에 실제 환경 변수 값으로 수정

In [ ]:
import os, boto3, botocore

ENDPOINT = "https://objectstorage.kr-central-2.kakaocloud.com"
REGION   = "kr-central-2"
BUCKET   = "models" # 버킷 이름
PREFIX   = "gender_predict" # 버킷 안 폴더 이름
LOCAL    = "./models/gender/model.joblib" # notebook의 모델 경로

AWS_ACCESS_KEY_ID = "{S3_ACCESS_KEY}"
AWS_SECRET_ACCESS_KEY = "{S3_SECRET_KEY}"

session = boto3.session.Session()
s3 = session.client(
    "s3",
    endpoint_url=ENDPOINT,
    region_name=REGION,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    config=botocore.config.Config(
        s3={"addressing_style": "path"},
        signature_version="s3v4",
    ),
)

# 버킷 존재 확인/생성
try:
    s3.head_bucket(Bucket=BUCKET)
    print(f"Bucket '{BUCKET}' already exists.")
except Exception:
    try:
        s3.create_bucket(Bucket=BUCKET)
    except Exception:
        s3.create_bucket(
            Bucket=BUCKET,
            CreateBucketConfiguration={"LocationConstraint": REGION}
        )
    print(f"Bucket '{BUCKET}' created.")

# 업로드
assert os.path.exists(LOCAL), f"로컬 파일 없음: {LOCAL}"
key = f"{PREFIX}/model.joblib"
s3.upload_file(LOCAL, BUCKET, key)
print(f"Uploaded: s3://{BUCKET}/{key}")

# 목록 확인
resp = s3.list_objects_v2(Bucket=BUCKET, Prefix=f"{PREFIX}/")
for obj in resp.get("Contents", []):
    print(obj["Key"], obj["Size"])

# 이후 Bash 셀에서 재사용할 수 있도록 .env 파일 생성
with open("kserve_s3_creds.env", "w", encoding="utf-8") as f:
    f.write(f'KEY_ID="{AWS_ACCESS_KEY_ID}"\n')
    f.write(f'SECRET="{AWS_SECRET_ACCESS_KEY}"\n')
print("Saved creds -> kserve_s3_creds.env")


## 3) KServe 리소스 생성
- KServe가 Object Storage에 있는 모델을 읽기 위한 리소스를 생성합니다.

In [ ]:
%%bash
set -euo pipefail

NAMESPACE=kbm-u-kubeflow-tutorial

source kserve_s3_creds.env
: "${KEY_ID:?KEY_ID missing}"
: "${SECRET:?SECRET missing}"

TS=$(date +%s)
SECRET_NAME="kakaos3-credentials-${TS}"
SA_NAME="sa-kakaos3-${TS}"

# 1) Secret 생성 (create만 사용)
cat <<EOF | kubectl create -n "$NAMESPACE" -f -
apiVersion: v1
kind: Secret
metadata:
  name: ${SECRET_NAME}
  annotations:
    serving.kserve.io/s3-endpoint: objectstorage.kr-central-2.kakaocloud.com
    serving.kserve.io/s3-region: kr-central-2
    serving.kserve.io/s3-usehttps: "1"
    serving.kserve.io/s3-verifyssl: "1"
    serving.kserve.io/s3-usevirtualbucket: "0"   # path-style 권장
type: Opaque
stringData:
  AWS_ACCESS_KEY_ID: "${KEY_ID}"
  AWS_SECRET_ACCESS_KEY: "${SECRET}"
EOF

# 2) ServiceAccount 생성 (Secret 참조)
cat <<EOF | kubectl create -n "$NAMESPACE" -f -
apiVersion: v1
kind: ServiceAccount
metadata:
  name: ${SA_NAME}
secrets:
  - name: ${SECRET_NAME}
EOF

# 다음 셀에서 쓰도록 파일로 저장
echo "SA_NAME=${SA_NAME}" > kserve_sa.env
echo "SECRET_NAME=${SECRET_NAME}" >> kserve_sa.env

echo "Created: SA=${SA_NAME}, SECRET=${SECRET_NAME} in ns=${NAMESPACE}"


## 4) KServe InferenceService 생성 및 모델 배포
- KServe InferenceService를 이용하여 모델을 배포합니다.

In [ ]:
%%bash
set -euo pipefail

NAMESPACE=kbm-u-kubeflow-tutorial
BUCKET='models'
PREFIX='gender_predict'

source kserve_sa.env
: "${SA_NAME:?SA_NAME missing}"

cat <<EOF | kubectl create -n "$NAMESPACE" -f -
apiVersion: serving.kserve.io/v1beta1
kind: InferenceService
metadata:
  name: gender-predict
spec:
  predictor:
    sklearn:
      protocolVersion: v2
      storageUri: s3://${BUCKET}/${PREFIX}
      resources:
        requests:
          cpu: "500m"
          memory: "1Gi"
        limits:
          cpu: "1"
          memory: "2Gi"
    serviceAccountName: ${SA_NAME}
EOF

# 모델 성별 추론 실습
### 배포된 모델에 샘플 데이터를 입력하여 성별을 추론하는 실습입니다.

## 5) 성별 추론
- 10개의 샘플 데이터를 입력하여 모델이 추론한 값을 확인합니다.

In [ ]:
import requests, json
import pandas as pd
import numpy as np
from IPython.display import display

URL = "http://gender-predict.kbm-u-kubeflow-tutorial.svc.cluster.local/v2/models/gender-predict/infer" # 배포된 모델 API URL

LABEL_MAP = {0: "F", 1: "M"}
TRUTH_MAP = {"여자": "F", "남자": "M"} 

#####################
# 10개의 샘플 데이터
#####################
cases = [
    {"case": "29세 남자", "truth": "남자",
     "data": [11,0,0,6.0,1.2727272727272727,7,1,0,3,0,0,1,0,0,0.0,0.0,0.09090909090909091,0.0,0.0,0.0,0.0,0.6931471805599453,0.0,0.0], "note": ""},
    {"case": "68세 여자", "truth": "여자",
     "data": [10,0,0,5.5,1.3,6,0,16,2,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0], "note": ""},
    {"case": "56세 여자", "truth": "여자",
     "data": [9,0,0,5.0,1.2222222222222223,6,2,20,2,2,0,0,0,1,0.2222222222222222,0.0,0.0,0.0,0.1111111111111111,1.0986122886681096,0.0,0.0,0.0,0.6931471805599453], "note": ""},
    {"case": "44세 남자", "truth": "남자",
     "data": [9,0,0,5.0,1.5555555555555556,4,0,17,2,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0], "note": ""},
    {"case": "28세 여자", "truth": "여자",
     "data": [8,0,0,4.5,1.25,6,1,20,2,0,0,0,1,0,0.0,0.0,0.0,0.125,0.0,0.0,0.0,0.0,0.6931471805599453,0.0], "note": ""},
    {"case": "68세 남자", "truth": "남자",
     "data": [6,0,0,3.5,1.3333333333333333,5,0,22,2,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0], "note": ""},
    {"case": "70세 여자", "truth": "여자",
     "data": [6,0,0,3.5,1.3333333333333333,5,1,19,2,0,0,0,0,1,0.0,0.0,0.0,0.0,0.16666666666666666,0.0,0.0,0.0,0.0,0.6931471805599453], "note": ""},
    {"case": "43세 남자", "truth": "남자",
     "data": [5,0,0,3.0,0.8,4,0,23,2,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
     "note": "검색/상품조회/카테고리 접근 없음"},
    {"case": "29세 여자", "truth": "여자",
     "data": [6,0,0,3.5,1.3333333333333333,4,0,2,3,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
     "note": "남성향 카테고리 활동 위주"},
    {"case": "41세 여자", "truth": "여자",
     "data": [2,0,0,1.5,0.5,2,0,20,2,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
     "note": "남성향/여성향 고르게 활동"},
]

def infer_one(vec):
    payload = {
        "inputs": [{
            "name": "input-0",
            "shape": [1, len(vec)],
            "datatype": "FP32",
            "data": vec,
        }]
    }
    r = requests.post(URL, json=payload, timeout=10)
    r.raise_for_status()
    js = r.json()
    if "outputs" in js:
        pred_int = js["outputs"][0]["data"][0]
    elif "predictions" in js:
        pred_int = js["predictions"][0]
    else:
        raise ValueError(f"Unexpected response keys: {list(js.keys())}")
    return int(pred_int)

rows = []
for c in cases:
    y_hat_int = infer_one(c["data"])
    truth_mf = TRUTH_MAP.get(c["truth"], c["truth"])
    pred_mf  = LABEL_MAP.get(y_hat_int, y_hat_int)
    correct  = (pred_mf == truth_mf)
    rows.append({
        "truth": truth_mf,
        "pred":  pred_mf,
        "✓/✗":   "✓" if correct else "✗",
        "note":  c["note"],
    })

df = pd.DataFrame(rows, columns=["truth", "pred", "✓/✗", "note"])

df.index = range(1, len(df) + 1)

display(df)

acc = (df["✓/✗"] == "✓").mean()
print(f"\n정확도(샘플 {len(df)}개): {acc:.3f}")
